Top <font color="blue"></font> <a name='Top' />

In [1]:
# The following piece of code gives the opportunity to show multiple outputs
# in one cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# Colorful outputs
class bcolors:
    RED       = '\033[91m'
    OKBLUE    = '\033[94m'
    BOLD      = '\033[1m'
    UNDERLINE = '\033[4m'
    ENDC      = '\033[0m'

<a href=#ImpLibHlprFunc>Importing Libraries & Helper Functions </a><br>
<a href=#EDA>EDA</a><br>


<a href=#Top>Top</a>

Importing Libraries & Helper Functions<font color="blue"></font> <a name='ImpLibHlprFunc' />

In [2]:
# Importing necessary libraries

import numpy as np
import pandas as pd

import statsmodels.api as sm
#from sklearn.metrics import mean_squared_error
#from math import sqrt

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import linear_model, svm
#from sklearn.linear_model import LinearRegression, Lasso

from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV

from sklearn.preprocessing import StandardScaler, PolynomialFeatures

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn import metrics
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score #, r2_score

#from sklearn.svm import SVR
#from sklearn.ensemble import RandomForestRegressor
#from sklearn.model_selection import GridSearchCV

In [3]:
# helper functions
# 1) PrintSeparator to print lines of repeated characters passed as the second parameter for the number of lines passed in the first parameters
def PrintSeparator(nLines=1,ch='-'):
  for i in range(nLines):
    print(ch*100)

def cus_print(var):
  print("numpy_array of type:" , type(var))
  PrintSeparator(nLines=1)
  print(var)
  PrintSeparator(nLines=2,ch='=')

def cus_print_numpyarray(npAr, printArFlag=False):
  print("ndim:", npAr.ndim, ", shape: ",npAr.shape)
  PrintSeparator(nLines=1)
  if printArFlag:
    print(npAr)
  else:
    print("first_element=", npAr[0], "last elemnent=" , npAr[len(npAr)-1])

  PrintSeparator(nLines=2,ch='=')

In [4]:
# for each column, display column name & its distinct contents ONLY in case the column has less than 6 unique values
def get_columns_and_distinct_values(df,nunique_target=5):
  str_=""
  ret=[]
  for column in df:
      columnSeriesObj = df[column]
      nunique_this_clmn = columnSeriesObj.nunique()
      str_ = 'Column Name: ' + column + ', Column Type: ' + str(df[column].dtype)
      if nunique_this_clmn == 1:
          str_ = str_ + ', Column has one unique value only and can be dropped from the dataframe'
          ret.append(str_)
          ret.append("------------------------------------------------------------")
      elif nunique_this_clmn <= nunique_target:
        str_ = str_ + ', Column has ' + str(columnSeriesObj.nunique()) + ' unique value(s) : '
        for val in columnSeriesObj.unique():
          str_ = str_  + str(val) + ","
        str_ = str_[:len(str_)-1]
        ret.append(str_)
        ret.append("------------------------------------------------------------")
      else:
        str_ = str_ + ', Column has more than ' + str(nunique_target) + ' unique values'
      #ret.append(str_)
      #ret.append("==========================================================================")

  print("==========================================================================")
  columns_of_type_object = df.select_dtypes(include=['object']).columns
  print("*** Columns of type object are:", columns_of_type_object)
  print("==========================================================================")
  columns_of_a_single_unique_value = df.columns[df.nunique() == 1]
  print("*** Columns of a single unique value are:", columns_of_a_single_unique_value, " can be dropped")
  print("==========================================================================")
  return ret

EDA <font color="blue"></font> <a name='EDA' />

#### Titanic Data Description

Ref./Source: https://www.kaggle.com/c/titanic/data


***Passenger ID*** to identify the passenger, numerical feature (Passenger ID/Ticket Number).

***Survived*** is our label, as we can see is a binary feature, 1 if survived and 0 otherwise.

***Pclass*** is the Ticket class (1 = 1st (Upper), 2 = 2nd (Middle), 3 = 3rd (lower))

***Name *** is the name of the passenger

***Sex*** is the gender of the passenger

***Age*** is the age in years

***Sibsp*** is the number of siblings / spouses aboard the Titanic

***Parch*** is the number of parents / children aboard the Titanic

***Ticket*** is the ticket number

***Fare*** is the Passenger fare

***Cabin*** is the cabin number

***Embarked*** means Port of Embarkation. C = Cherbourg, Q = Queenstown, S = Southampton

Load data

In [5]:
# load train data from train.csv
titanic_train_df = pd.read_csv(r'titanic_train.csv')
titanic_train_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'titanic_train.csv'

In [ ]:
titanic_train_df.columns

##### EDA

In [ ]:
titanic_train_df.info()

In [ ]:
titanic_train_df.shape

In [ ]:
# check for NULL values
if titanic_train_df.isnull().any().sum() > 0:
    print("There are:" , titanic_train_df.isnull().any().sum(), "Null values in the train data")
else:
    print("Train data is good - There is no null values in any of the columns")

In [ ]:
titanic_train_df.describe(include = 'all')

In [ ]:
titanic_train_df.drop(['Name','Ticket','PassengerId'], axis=1, inplace=True)

In [ ]:
print ("titanic_train_df:")
#print("null values:\n", titanic_train_df.isnull().sum())
print("duplicated records", titanic_train_df.duplicated().sum())

###### Embarked Feature / Two Records with Nan values - replace the 2 NULL values in embarked with S = Southampton (as most passengers on Titanic embarked from this port)

In [ ]:
# Two records in titanic_train_df with Nan in Embarked feature  - will be dropped from the dataframe
titanic_train_df[titanic_train_df['Embarked'].isnull()]

In [ ]:
print(titanic_train_df['Embarked'].value_counts())
# most passengers are emabrked from S = Southampton (Three ports of Embarkation C = Cherbourg; Q = Queenstown; S = Southampton)

# replace the 2 NULL values in embarked with S = Southampton (as most passengers on Titanic embarked from this port)
titanic_train_df.loc[titanic_train_df['Embarked'].isnull(),'Embarked'] = 'S'

print(titanic_train_df['Embarked'].value_counts())

titanic_train_df[titanic_train_df['Embarked'].isnull()]

###### Cabin Feature / replace missing values in titanic_train_df['Cabin'] with letter 'U' for unknown

In [ ]:
print("null values in the Cabin Feature:\n", titanic_train_df.isnull().sum()['Cabin'], "Records")

In [ ]:
titanic_train_df['Cabin'].unique()

In [ ]:
# keep first letter only in the Cabin
titanic_train_df['Cabin'] = titanic_train_df['Cabin'].str[0]
titanic_train_df['Cabin'].unique()

In [ ]:
# replace missing values in titanic_train_df['Cabin'] with letter 'U' for unknown
titanic_train_df['Cabin'].fillna('U', inplace=True)

In [ ]:
titanic_train_df['Cabin'].unique()

###### Age Feature / replace missing values in titanic_train_df['Age'] with the avergae of age

In [ ]:
print("null values in the Age Feature:\n", titanic_train_df.isnull().sum()['Age'], "Records")

In [ ]:
# Fill missing/Nan values in Age with average age
titanic_train_df['Age'].fillna(titanic_train_df['Age'].mean(), inplace=True)  # 29.642093
print("null values in the Age Feature:\n", titanic_train_df.isnull().sum()['Age'], "Records")
#titanic_train_df['Age'].unique()


In [ ]:
# Check again after fixing missing records
titanic_train_df.info()

In [ ]:
titanic_train_df.describe(include = 'all')

In [ ]:
titanic_train_df.head()

In [ ]:
ret=get_columns_and_distinct_values(titanic_train_df,10)
for str_ in ret:
  print(str_)

###### Use data for ***Fare & Cabin*** to provide ***better/more accurate*** values for the "U = Unknown" cabins for some/many records

In [ ]:
# plot using box plot to check for outliers of Fare for the Unknown Cabins (Train df)
plt.figure(figsize=(15,5))
plt.title("Box plot for Fare per Cabin - Titanic Train df")
sns.boxplot(x='Cabin', y='Fare',data=titanic_train_df)
plt.show()

In [ ]:
def cbnFareDataToDataFrame(titanic_df):
  CntCbnFare, MinCbnFare, MaxCbnFare, AvgCbnFare, MedianCbnFare, ModeCbnFare = [],[],[],[],[],[]
  for cbn in titanic_df['Cabin'].unique(): #'ABCDEFGTU':
    condition = (titanic_df['Cabin'] == cbn) & (titanic_df['Fare']!=0)

    #strCbnFareInfo = "For Cabin: " + cbn

    cnt=titanic_df[condition]['Fare'].count()
    #strCbnFareInfo += " ===> Count=" + str(cnt)
    CntCbnFare.append(cnt)

    min_=round(titanic_df[condition]['Fare'].min(),2)
    #strCbnFareInfo += ", Min Fare=" + str(min)
    MinCbnFare.append(min_)

    max_=round(titanic_df[condition]['Fare'].max(),2)
    #strCbnFareInfo += ", Max=" + str(max_)
    MaxCbnFare.append(max_)

    Mean_=round(titanic_df[condition]['Fare'].mean(),2)
    #strCbnFareInfo += ", Mean=" + str(Mean_)
    AvgCbnFare.append(Mean_)

    Median_=round(titanic_df[condition]['Fare'].median(),2)
    #strCbnFareInfo += ", Median=" + str(Median_)
    MedianCbnFare.append(Median_)

    Mode_=round(titanic_df[condition]['Fare'].mode().mean(),2)
    #strCbnFareInfo += ", Mode=" + str(Mode_)
    ModeCbnFare.append(Mode_)

  #print(strCbnFareInfo)

  dict = {'Cbn': list(titanic_df['Cabin'].unique()), 'Count': CntCbnFare, 'Min': MinCbnFare, 'Max': MaxCbnFare, 'Mean': AvgCbnFare, 'Median': MedianCbnFare, 'Mode': ModeCbnFare}
  cbnFareDf = pd.DataFrame(dict)
  cbnFareDf.head(len(titanic_df['Cabin'].unique()))

  #print(cbnFareDf.sort_values(by=['Median','Mean', 'Mode'], ascending=[True,True, True]))
  return cbnFareDf

In [ ]:
cbnFareTrainDf = cbnFareDataToDataFrame(titanic_train_df)
#cbnFareDf.head(len('ABCDEFGTU'))
print(cbnFareTrainDf.sort_values(by=['Mean','Median','Mode'], ascending=[False,False, False]))

In [ ]:
# first obsevation (only for titanic train) that there is only one entry for cabin T so high probability
# it is a data-issue - will fix to Cabin A as same Mode & closed Median & Mean
titanic_train_df.loc[titanic_train_df['Cabin'] == 'T', 'Cabin'] = 'A'

cbnFareTrainDf = cbnFareDataToDataFrame(titanic_train_df)
print(cbnFareTrainDf.sort_values(by=['Mean','Median','Mode'], ascending=[False,False, False]))

In [ ]:
# second obsevation that there are many entries for cabin U which has many outliers (also from the above boxplot)
# so fix this data issue by assigning a different cabin based on fare Mean
#print(cbnFareTrainDf.sort_values(by=['Max'], ascending=[False]))

In [ ]:
cndTrain1 = (titanic_train_df['Cabin'] == 'U')

cndTrain2 = (titanic_train_df['Fare'] > 100.15)
titanic_train_df.loc[cndTrain1 & cndTrain2, 'Cabin'] = 'B'

cndTrain2 = (titanic_train_df['Fare'] <= 100.15) & (titanic_train_df['Fare'] > 57.24)
titanic_train_df.loc[cndTrain1 & cndTrain2, 'Cabin'] = 'C'

cndTrain2 = (titanic_train_df['Fare'] <= 57.24) & (titanic_train_df['Fare'] > 46.03)
titanic_train_df.loc[cndTrain1 & cndTrain2, 'Cabin'] = 'D'

cndTrain2 = (titanic_train_df['Fare'] <= 46.03) & (titanic_train_df['Fare'] > 41.99)
titanic_train_df.loc[cndTrain1 & cndTrain2, 'Cabin'] = 'E'

cndTrain2 = (titanic_train_df['Fare'] <= 41.99) & (titanic_train_df['Fare'] > 18.70)
titanic_train_df.loc[cndTrain1 & cndTrain2, 'Cabin'] = 'A'

cndTrain2 = (titanic_train_df['Fare'] <= 18.70) & (titanic_train_df['Fare'] > 13.58)
titanic_train_df.loc[cndTrain1 & cndTrain2, 'Cabin'] = 'F'

cndTrain2 = (titanic_train_df['Fare'] <= 13.58) & (titanic_train_df['Fare'] > 7.65)
titanic_train_df.loc[cndTrain1 & cndTrain2, 'Cabin'] = 'G'

#print("For Training Data")
cbnFareTrainDf = cbnFareDataToDataFrame(titanic_train_df)
print(cbnFareTrainDf.sort_values(by=['Max'], ascending=[False]))

In [ ]:
# plot using box plot to check for Fare per Cabin after above step (Train)
plt.figure(figsize=(15,5))
plt.title("Box plot for Fare per Cabin")
sns.boxplot(x='Cabin', y='Fare',data=titanic_train_df)
plt.show()

In [ ]:
titanic_train_df2 = pd.get_dummies(titanic_train_df)
titanic_train_df2.head()

### **C.1.2. Data split: Traing set & Test set**

In [ ]:
X_tnc_train=titanic_train_df2.drop(['Survived'],axis=1)
y_tnc_train=titanic_train_df2['Survived']

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_tnc_train,y_tnc_train,test_size=0.2)

# scale data separately for train and test
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

### **C.1.3. Applying Random Forest**

In [ ]:
rfc= RandomForestClassifier(n_estimators=100,random_state=47)
rfc.fit(X_train_scaled,y_train)
y_train_predict_rfc=rfc.predict(X_train_scaled)

print("train accuracy_score", accuracy_score(y_train, y_train_predict_rfc))
print("train f1_score", f1_score(y_train, y_train_predict_rfc, average="macro"))
print("train precision_score",precision_score(y_train, y_train_predict_rfc, average="macro"))   # "macro" ---> does not take into account imbalance
print("train recall_score",recall_score(y_train, y_train_predict_rfc, average="macro"))
print("test rfc score",rfc.score(X_train_scaled,y_train))

In [ ]:
y_test_predict_rfc = rfc.predict(X_test_scaled)
print("test accuracy_score", accuracy_score(y_test, y_test_predict_rfc))
print("test f1_score",f1_score(y_test, y_test_predict_rfc, average="macro"))
print("test precision_score",precision_score(y_test, y_test_predict_rfc, average="macro"))   # "macro" ---> does not take into account imbalance
print("test recall_score",recall_score(y_test, y_test_predict_rfc, average="macro"))
print("test rfc score",rfc.score(X_test_scaled,y_test))

In [ ]:
figsize = plt.figure(figsize = (12,8))
# Initialize the accuracy_score vector
acc = []              # test accuracy
acc_train = []        # train accuracy
estimators = 2 ** np.arange(9)
print(estimators)

# Calculate accuracy score on the test set for different values for the n_estimators parameter
for est in estimators:
    # Fit the Regression Tree
    rfc2 = RandomForestClassifier(n_estimators=est, random_state=47) #max_depth=i,
    rfc2.fit(X_train_scaled,y_train)
    # Predict on the test set
    y_pred = rfc2.predict(X_test_scaled)
    # Compute the accuracy
    score = accuracy_score(y_test, y_pred)
    acc.append(score)
    acc_train.append(accuracy_score(y_train, rfc2.predict(X_train_scaled)))

# Plot results
#print(acc)
#print(acc_train)
plot = plt.plot(estimators, acc, '-', estimators, acc_train, 'r')
#plot = plt.plot(estimators, acc_train, 'b', label='Train')
#plot = plt.plot(estimators, acc, 'r', label='Test')
xlab = plt.xlabel('# estimators', fontsize = 17)
ylab = plt.ylabel('Accuracy', fontsize = 17)
plt.legend(['Train','Test'])
title = plt.title('Random Forest - Accuracy vs Depth of Trees (Titanic)', fontsize = 20);

#### ***Random Forest Accuracy Results***

In [ ]:
# best value for estimators:
print("Using Random forest with different number of estimators, best accuracy was at", max(acc),"and it occurs when estimators=", estimators[acc.index(max(acc))])

### **C.1.4. Applying SVM**

In [ ]:
svc=SVC(C=1000) #kernel='rbf',
svc.fit(X_train_scaled,y_train)

In [ ]:
y_train_predict_svc=svc.predict(X_train_scaled)

print("train accuracy_score", accuracy_score(y_train, y_train_predict_svc))
print("train f1_score", f1_score(y_train, y_train_predict_svc, average="macro"))
print("train precision_score",precision_score(y_train, y_train_predict_svc, average="macro"))   # "macro" ---> does not take into account imbalance
print("train recall_score",recall_score(y_train, y_train_predict_svc, average="macro"))
print("test svc score",svc.score(X_train_scaled,y_train))

In [ ]:
y_test_predict_svc = svc.predict(X_test_scaled )
print("test accuracy_score", accuracy_score(y_test, y_test_predict_svc))
print("test f1_score",f1_score(y_test, y_test_predict_svc, average="macro"))
print("test precision_score",precision_score(y_test, y_test_predict_svc, average="macro"))   # "macro" ---> does not take into account imbalance
print("test recall_score",recall_score(y_test, y_test_predict_svc, average="macro"))
print("test svc score",svc.score(X_test_scaled,y_test))

In [ ]:
figsize = plt.figure(figsize = (12,8))
# Initialize the accuracy_score vector
acc = []              # test accuracy
acc_train = []        # train accuracy
reglCs = 2 ** np.arange(9)
print(reglCs)

# Calculate accuracy score on the test set for different values for the regularization parameter C
for C_param in reglCs:
    # Fit the Regression Tree
    svc2 = SVC(C=C_param) #max_depth=i,
    svc2.fit(X_train_scaled,y_train)
    # Predict on the test set
    y_pred = svc2.predict(X_test_scaled)
    # Compute the accuracy
    score = accuracy_score(y_test, y_pred)
    acc.append(score)
    acc_train.append(accuracy_score(y_train, svc2.predict(X_train_scaled)))

# Plot results
#print(acc)
#print(acc_train)
plot = plt.plot(reglCs, acc, '-', reglCs, acc_train, 'r')
#plot = plt.plot(reglCs, acc_train, 'b', label='Train')
#plot = plt.plot(reglCs, acc, 'r', label='Test')
xlab = plt.xlabel('Reg. Parameter C', fontsize = 17)
ylab = plt.ylabel('Accuracy', fontsize = 17)
plt.legend(['Train','Test'])
title = plt.title('SVM - Accuracy vs Depth of Trees (Titanic)', fontsize = 20);

#### ***SVM Accuracy Results***

In [ ]:
# best value for estimators:
print("Using SVM with different values for the regularization parameter C, best accuracy was at", max(acc),"and it occurs when C=", reglCs[acc.index(max(acc))])